In [2]:
!pip install gensim
!pip install nltk
!pip install numpy
!pip install pandas
!pip install re
!pip install spacy
!pip install sklearn

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [3]:
pip install tidytext

  Using cached tidytext-0.0.1.tar.gz (4.3 kB)
  Using cached siuba-0.0.25-py3-none-any.whl (112 kB)
  Created wheel for tidytext: filename=tidytext-0.0.1-py3-none-any.whl size=3905 sha256=048321031fa96879618963c3ea1fe8494c855b7c04c3cddbfd30c2ab28a023c4
  Stored in directory: c:\users\yusuf\appdata\local\pip\cache\wheels\07\03\c0\f73eeef462dd66dbca0288a338fcbcdc78e3588937ccc907d8
Successfully built tidytext
Note: you may need to restart the kernel to use updated packages.


In [1]:
import guidedlda
import sklearn
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import joblib
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from nltk.tokenize import word_tokenize
np.random.seed(42)

E:\Program_files\Conda\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yusuf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Yusuf\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
papers = pd.read_csv('data.csv')
# Print head
papers.head()

,text
0,Tires where delivered to the garage of my choi...
1,"Easy Tyre Selection Process, Competitive Prici..."
2,Very easy to use and good value for money.
3,Really easy and convenient to arrange
4,It was so easy to select tyre sizes and arrang...


In [4]:
from platform import python_version

print(python_version())

3.8.3


In [5]:
# Load the regular expression library
import re

# Remove punctuation
papers['paper_text_processed'] = papers['text'].map(lambda x: re.sub('[,\.!?]', ' ', x))
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))
# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers['paper_text_processed'].head()

0    tires where delivered to the garage of my choi...
1    easy tyre selection process  competitive prici...
2           very easy to use and good value for money 
3                really easy and convenient to arrange
4    it was so easy to select tyre sizes and arrang...
Name: paper_text_processed, dtype: object

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['tires', 'where', 'delivered', 'to', 'the', 'garage', 'of', 'my', 'choice', 'the', 'garage', 'notified', 'me', 'when', 'they', 'had', 'been', 'delivered', 'day', 'and', 'time', 'was', 'arranged', 'with', 'the', 'garage', 'and', 'went', 'and', 'had']


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yusuf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-07-25 21:51:44.758085: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-07-25 21:51:44.758486: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['tire', 'deliver', 'garage', 'choice', 'garage', 'notify', 'deliver', 'day', 'time', 'arrange', 'garage', 'go', 'fit', 'hassel', 'free', 'experience']


In [12]:
import gensim.corpora as corpora

# Create Dictionary
dictionary = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]


In [13]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 arrange
1 choice
2 day
3 deliver
4 experience
5 fit
6 free
7 garage
8 go
9 hassel
10 notify


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [18]:
#bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

NameError: name 'processed_docs' is not defined

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=10, workers=2, alpha='symmetric',eta=0.91)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [15]:
def get_wordnet_pos(word):
    '''tags parts of speech to tokens
    Expects a string and outputs the string and 
    its part of speech'''
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
def word_lemmatizer(text):
    '''lemamtizes the tokens based on their part of speech'''
    
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text, get_wordnet_pos(text))
    return text
def reflection_tokenizer(text):
    '''expects a string an returns a list of lemmatized tokens 
    and removes the stop words. Tokens are lower cased and 
    non- alphanumeric characters as well as numbers removed. '''
    text=re.sub(r'[\W_]+', ' ', text) #keeps alphanumeric characters
    text=re.sub(r'\d+', '', text) #removes numbers
    text = text.lower()
    tokens = [word for word in word_tokenize(text)]
    tokens = [word for word in tokens if len(word) >= 3]
    #removes smaller than 3 character
    tokens = [word_lemmatizer(w) for w in tokens]
    tokens = [s for s in tokens if s not in stop_words]
    return tokens

In [16]:
papers['lemmatize_token'] = papers.text.apply(reflection_tokenizer)

In [17]:
papers.head()

,text,paper_text_processed,lemmatize_token
0,Tires where delivered to the garage of my choi...,tires where delivered to the garage of my choi...,"[tire, deliver, garage, choice, garage, notify..."
1,"Easy Tyre Selection Process, Competitive Prici...",easy tyre selection process competitive prici...,"[easy, tyre, selection, process, competitive, ..."
2,Very easy to use and good value for money.,very easy to use and good value for money,"[easy, use, good, value, money]"
3,Really easy and convenient to arrange,really easy and convenient to arrange,"[really, easy, convenient, arrange]"
4,It was so easy to select tyre sizes and arrang...,it was so easy to select tyre sizes and arrang...,"[easy, select, tyre, size, arrange, local, fit..."


In [18]:
papers.to_csv('data_lemmatized')

In [19]:
from nltk.corpus import stopwords
stoplist = set(stopwords.words("english"))

In [20]:
token_vectorizer = CountVectorizer(tokenizer = reflection_tokenizer, min_df=10, stop_words=stoplist, ngram_range=(1, 4))

In [21]:
X_ngrams = token_vectorizer.fit_transform(papers.paper_text_processed)

E:\Program_files\Conda\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [22]:
type(X_ngrams)

scipy.sparse.csr.csr_matrix

In [23]:
X_ngrams.shape

(10132, 2570)

In [24]:
tf_feature_names = token_vectorizer.get_feature_names()

In [25]:
word2id = dict((v, idx) for idx, v in enumerate(tf_feature_names))

In [26]:
seed_topic_list_6 = [['cost', 'effective', 'expensive', 'money', 'value', 'price', 'cheap','good price', 'reasonable','pricing'],
                   ['quick', 'efficient', 'great service', 'service','fast', 'good service', 'excellent', 'professional', 'rude', 'bad','service'],
                   ['easy','appointment', 'book','schedule'],
                   ['tyre', 'great', 'finish', 'good', 'awesome', 'excellent', 'fitting', 'reliable'],
                   ['fitter', 'mobile', 'mechanic', 'staff', 'friendly','polite'],
                   ['area', 'near', 'far','location'],
                   ['wait', 'hour', 'time','fast' ],
                   ['deliver', 'order','timely'],
                   ['confusion', 'rebook'],
                   ['wait','long', 'waste', 'time'],
                   ['discount', 'offer', 'price'],
                   ['reschedule', 'rebook', 'change', 'date']]

In [27]:
seed_dict = {'Value for Money' : 0, 'garage service' : 1, 'ease of booking' : 2 , 'tyre quality' : 3, 'mobile fitter' : 4,
             'location' : 5, 'length of fitting' : 6, 'delivery punctuality' : 7, 'booking confusion' : 8, 'wait time' : 9, 'discounts' :
              10, 'change of date': 11}

In [28]:
seed_topics = {}

In [29]:
seed_topics.update({
    word2id["cost"] : seed_dict["Value for Money"], word2id["effective"] : seed_dict["Value for Money"], 
    word2id["expensive"] : seed_dict["Value for Money"], word2id["money"] : seed_dict["Value for Money"], 
    word2id["value"] : seed_dict["Value for Money"], word2id["price"] : seed_dict["Value for Money"], 
    word2id["cheap"] : seed_dict["Value for Money"], word2id["good price"] : seed_dict["Value for Money"],
    word2id["reasonable"] : seed_dict["Value for Money"], word2id["pricing"] : seed_dict["Value for Money"]
})

In [30]:
seed_topics.update({
    word2id["quick"] : seed_dict["garage service"], word2id["efficient"] : seed_dict["garage service"], 
    word2id["great service"] : seed_dict["garage service"], word2id["service"] : seed_dict["garage service"], 
    word2id["fast"] : seed_dict["garage service"], word2id["good service"] : seed_dict["garage service"], 
    word2id["excellent"] : seed_dict["garage service"], word2id["professional"] : seed_dict["garage service"],
    word2id["rude"] : seed_dict["garage service"], word2id["bad"] : seed_dict["garage service"], 
    word2id["service"] : seed_dict["garage service"]
})

In [31]:
seed_topics.update({
    word2id["easy"] : seed_dict["ease of booking"], word2id["appointment"] : seed_dict["ease of booking"], 
    word2id["book"] : seed_dict["ease of booking"], word2id["schedule"] : seed_dict["ease of booking"]
})

In [32]:
seed_topics.update({
    word2id["tyre"] : seed_dict["tyre quality"], word2id["great"] : seed_dict["tyre quality"], 
    word2id["finish"] : seed_dict["tyre quality"], word2id["good"] : seed_dict["tyre quality"],
    word2id["awesome"] : seed_dict["tyre quality"], word2id["excellent"] : seed_dict["tyre quality"],
    word2id["fitting"] : seed_dict["tyre quality"], word2id["reliable"] : seed_dict["tyre quality"]
})

In [33]:
seed_topics.update({
    word2id["fitter"] : seed_dict["mobile fitter"], word2id["mobile"] : seed_dict["mobile fitter"], 
    word2id["mechanic"] : seed_dict["mobile fitter"], word2id["staff"] : seed_dict["mobile fitter"],
    word2id["friendly"] : seed_dict["mobile fitter"], word2id["polite"] : seed_dict["mobile fitter"]
})

In [34]:
seed_topics.update({
    word2id["area"] : seed_dict["location"], word2id["near"] : seed_dict["location"], 
    word2id["far"] : seed_dict["location"], word2id["location"] : seed_dict["location"]
})

In [35]:
seed_topics.update({
    word2id["wait"] : seed_dict["length of fitting"], word2id["hour"] : seed_dict["length of fitting"], 
    word2id["time"] : seed_dict["length of fitting"], word2id["fast"] : seed_dict["length of fitting"]
})

In [36]:
seed_topics.update({
    word2id["deliver"] : seed_dict["delivery punctuality"], word2id["order"] : seed_dict["delivery punctuality"], 
    word2id["timely"] : seed_dict["delivery punctuality"]
})

In [37]:
seed_topics.update({
    word2id["confusion"] : seed_dict["booking confusion"], word2id["rebook"] : seed_dict["booking confusion"]
})

In [38]:
seed_topics.update({
    word2id["wait"] : seed_dict["wait time"], word2id["long"] : seed_dict["wait time"], 
    word2id["time"] : seed_dict["wait time"], word2id["waste"] : seed_dict["wait time"]
})

In [39]:
seed_topics.update({
    word2id["discount"] : seed_dict["discounts"], word2id["offer"] : seed_dict["discounts"], 
    word2id["price"] : seed_dict["discounts"]
})

In [40]:
seed_topics.update({
    word2id["reschedule"] : seed_dict["change of date"], word2id["rebook"] : seed_dict["change of date"], 
    word2id["change"] : seed_dict["change of date"], word2id["date"] : seed_dict["change of date"]
})

In [41]:
from guidedlda import GuidedLDA as lda

In [43]:
model = lda(n_topics= 12, n_iter=1000, random_state=1, refresh=50)
model.fit(X_ngrams, seed_topics = seed_topics, seed_confidence = 0.3)
topic_word = model.topic_word_
n_top_words = 25
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(tf_feature_names)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:guidedlda:n_documents: 10132
INFO:guidedlda:vocab_size: 2570
INFO:guidedlda:n_words: 163366
INFO:guidedlda:n_topics: 12
INFO:guidedlda:n_iter: 1000
INFO:guidedlda:<0> log likelihood: -1860300
INFO:guidedlda:<50> log likelihood: -1119680
INFO:guidedlda:<100> log likelihood: -1101654
INFO:guidedlda:<150> log likelihood: -1094332
INFO:guidedlda:<200> log likelihood: -1090766
INFO:guidedlda:<250> log likelihood: -1088768
INFO:guidedlda:<300> log likelihood: -1087419
INFO:guidedlda:<350> log likelihood: -1085991
INFO:guidedlda:<400> log likelihood: -1085749
INFO:guidedlda:<450> log likelihood: -1085369
INFO:guidedlda:<500> log likelihood: -1084287
INFO:guidedlda:<550> log likelihood: -1084744
INFO:guidedlda:<600> log likelihood: -1083924
INFO:guidedlda:<650> log likelihood: -1083784
INFO:guidedlda:<700> log likelihood: -1083454
INFO:guidedlda:<750> log likelihood: -1082671
INFO:guidedlda:<800> log likelihood: -1082780
INFO:guidedlda:<850> log likelihood: -1082146
INFO:guidedlda:<900> l

Topic 0: value good service easy good value money great value money great value quick efficient excellent use tyre way simple buy save quick easy excellent value good value money way buy effective great service easy use
Topic 1: service great price great service excellent great price efficient customer excellent service service great customer service fast friendly competitive use service great price efficient service quick easy price great good service easy always great service great fantastic
Topic 2: easy use price tyre website easy use good choice fitting great book garage local use website easy use website excellent site competitive selection choose range price easy fitter competitive price website easy
Topic 3: price good service tyre excellent great fitting good price choice excellent service quality great price garage tyre good fit choice tyre good choice price excellent excellent price centre tyre great price tyre quality tyre price good great service
Topic 4: tyre fit car gara

In [44]:
doc_topic = model.transform(X_ngrams)
print(doc_topic)

[[3.41420214e-05 1.35995332e-05 2.60199922e-03 ... 1.49376327e-01
  8.78460807e-05 5.69230088e-02]
 [7.60890350e-04 1.22685140e-03 4.76738801e-01 ... 1.18087222e-04
  8.63175408e-04 4.02596157e-04]
 [9.92441354e-01 1.47019383e-04 2.41009353e-03 ... 3.14606630e-05
  3.32795266e-03 1.60815234e-04]
 ...
 [1.13717283e-04 3.90809695e-05 2.79728421e-04 ... 5.17186447e-03
  1.68268351e-01 3.89153637e-01]
 [4.33170017e-04 6.34722554e-01 6.34345550e-04 ... 3.45378364e-03
  8.92604203e-04 7.40833658e-04]
 [6.05681489e-06 1.46229230e-04 9.76687633e-04 ... 8.29760797e-03
  3.06448589e-03 8.97674623e-04]]


In [45]:
doc_topic[1]

array([7.60890350e-04, 1.22685140e-03, 4.76738801e-01, 3.99776131e-01,
       1.27955238e-04, 5.31552156e-04, 3.19315036e-04, 1.18882062e-01,
       2.52582438e-04, 1.18087222e-04, 8.63175408e-04, 4.02596157e-04])

In [46]:
columns_label = ['topic {}'.format(i) for i in range(12)]  # number of topics
topic_vector = pd.DataFrame(doc_topic, columns = columns_label)#dataframe of doc-topics
topic_vector.round(2).head(10)

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,topic 10,topic 11
0,0.00,0.00,0.00,0.00,0.01,0.00,0.0,0.75,0.03,0.15,0.00,0.06
1,0.00,0.00,0.48,0.40,0.00,0.00,0.0,0.12,0.00,0.00,0.00,0.00
2,0.99,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
3,0.52,0.00,0.00,0.00,0.00,0.02,0.0,0.45,0.00,0.00,0.00,0.00
4,0.00,0.00,0.88,0.00,0.00,0.00,0.0,0.11,0.00,0.00,0.00,0.00
5,0.00,0.32,0.00,0.00,0.01,0.00,0.0,0.00,0.00,0.65,0.00,0.00
6,0.00,0.29,0.38,0.30,0.00,0.00,0.0,0.00,0.00,0.00,0.02,0.00
7,0.00,0.00,0.00,0.99,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
8,0.00,0.01,0.10,0.00,0.01,0.00,0.0,0.02,0.00,0.00,0.02,0.83
9,0.00,0.00,0.13,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.85,0.00


In [47]:
def topic_threshold(doc_topic, topic_vector):
    
    """
    Return the topic number if the probablity of a topic being in a document is more than value
    """
    
    topic_num_list = []
    for i in range(len(topic_vector)):
        topic_num = [idx for idx, value in enumerate(doc_topic[i]) if value >= 0.25]
        if topic_num != []:
            topic_num = topic_num
        else:
            topic_num = 'None'
        topic_num_list.append(topic_num)
    return topic_num_list

In [48]:
num_topic=topic_threshold(doc_topic, topic_vector)
#print(num_topic)

In [49]:
df_doc_topic = pd.DataFrame({'topics': num_topic, 'reflection': papers.paper_text_processed})

In [53]:
df_doc_topic.to_csv("ded.csv")